# Week 1

In [ ]:
#~source: https: //en.wikipedia.org/wiki/Gradient_descent
# code source: https://en.wikipedia.org/w/index.php?title=Gradient_descent&oldid=966271567

# 初始值
next_x = 6# We start the search at x = 6
# 步长系数（学习率）
gamma = 0.01# Step size multiplier
# 提前停止（系数变化小于此值时停止）
precision = 0.00001# Desired precision of result
# 最大迭代次数
max_iters = 10000# Maximum number of iterations

# Derivative
#function
#求导
def df(x):
  return 4 * x ** 3 - 9 * x ** 2

# 迭代
for i in range(max_iters):
    current_x = next_x
    
    #梯度下降
    next_x = current_x - gamma * df(current_x)
    print(i, next_x, df(current_x))

    # 提前停止的判定，计算系数变化了多少，小于阈值后提前停止
    step = next_x - current_x
    if abs(step) <= precision:
        break

print("Minimum at ", next_x)

# The output for the above will be something like 
# "Minimum at 2.2499646074278457"

In [ ]:
# Source: https://github.com/mattnedrich/GradientDescentExample
# y = mx + b
# m is slope, b is y-intercept
# 计算均方误差 MSE
def compute_error_for_line_given_points(b, m, points):
    totalError = 0
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        # 实际的y减去 mx+b 的 y
        totalError += (y - (m * x + b)) ** 2
    return totalError / float(len(points))

In [ ]:
# Source: https://github.com/mattnedrich/GradientDescentExample

# 梯度下降用于线性回归
def step_gradient(b_current, m_current, points, learningRate):
    b_gradient = 0
    m_gradient = 0
    N = float(len(points))
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        
        # 负梯度计算 （Loss Function 对 y_pre的导数，乘以 y_pred 对 m和b的导数
        # 除以N是因为有N个样本，相当于取平均值
        b_gradient += -(2/N) * (y - ((m_current * x) + b_current))
        m_gradient += -(2/N) * x * (y - ((m_current * x) + b_current))
    new_b = b_current - (learningRate * b_gradient)
    new_m = m_current - (learningRate * m_gradient)
    return [new_b, new_m]

In [ ]:
# 执行梯度下降的函数
def gradient_descent_runner(points, starting_b, starting_m, learning_rate, num_iterations):
    b = starting_b
    m = starting_m
    for i in range(num_iterations):
        b, m = step_gradient(b, m, array(points), learning_rate)
    return [b, m]

In [ ]:
# 用上面的代码合到一起，跑起来
def run():
    points = genfromtxt("data_linearreg.csv", delimiter=",")
    learning_rate = 0.0001
    initial_b = 0 # initial y-intercept guess
    initial_m = 0 # initial slope guess
    num_iterations = 1000
    print ("Starting gradient descent at b = {0}, m = {1}, error = {2}".format(initial_b, initial_m, compute_error_for_line_given_points(initial_b, initial_m, points)))
    print ("Running...")
    [b, m] = gradient_descent_runner(points, initial_b, initial_m, learning_rate, num_iterations)
    print ("After {0} iterations b = {1}, m = {2}, error = {3}".format(num_iterations, b, m, compute_error_for_line_given_points(b, m, points)) )

In [ ]:
#source: https://stackoverflow.com/questions/3949226/calculating-pearson-correlation-and-significance-in-python

import math
import numpy as np
from random import random

# 计算 R Score
def pcc(X, Y):
   ''' Compute Pearson Correlation Coefficient. '''
   # Normalise X and Y
   X -= X.mean(0)
   Y -= Y.mean(0)
   # Standardise X and Y
   X /= X.std(0)
   Y /= Y.std(0)
   # Compute mean product
   return np.mean(X*Y)
 
def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    # 协方差 除以 两个标准差的乘积
    return diffprod / math.sqrt(xdiff2 * ydiff2)

#main

# Using it on a random example

X = np.array([random() for x in range(100)])
Y = np.array([random() for x in range(100)])


# 两种等效的写法
pcof = pcc(X, Y)
print(pcof, ' is pcof')
 
pcoftwo = pearson_def(X, Y)
print(pcoftwo, ' is pcof second version')

# Week 2

In [ ]:
#Source: https://machinelearningmastery.com/implement-logistic-regression-stochastic-gradient-descent-scratch-python/

from math import exp

# Make a prediction with coefficients
# 预测输出的概率 
# 先计算 y_hat， 再使用 sigmoid函数转化为概率
def predict(row, coefficients):
	yhat = coefficients[0]
	for i in range(len(row)-1):
		yhat += coefficients[i + 1] * row[i]
	return 1.0 / (1.0 + exp(-yhat))

# Estimate logistic regression coefficients using stochastic gradient descent
# 梯度下降 更新参数 (权重 w 和 b) 
# 虽然标注了SGD，但这不是一个随机梯度下降，而是使用 SSE作为损失函数的GD： （全量的）批量梯度下降（如果使用MSE，求导时也要取均值，乘以1/N）
def coefficients_sgd(train, l_rate, n_epoch):
    # 初始化系数
	coef = [0.0 for i in range(len(train[0]))]
    # 迭代过程
	for epoch in range(n_epoch):
		sum_error = 0
        # 循环 遍历每一个sample
		for row in train:
            # 计算 预测的prop
			yhat = predict(row, coef)
            # 计算 伪残差
			error = row[-1] - yhat
            # 计算 伪残差平方和，类似MSE
			sum_error += error**2
            # 在原有的 系数基础上，加上 负梯度乘以学习率乘以 sigmoid 的导数
            # 这里是在更新 b(bias) wx+b的b
			coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
			#print(row, yhat, error, sum_error)
            # 这里是更新 w
			for i in range(len(row)-1):
				coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
	return coef

# Calculate coefficients
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
l_rate = 0.3
n_epoch = 10
coef = coefficients_sgd(dataset, l_rate, n_epoch)
print(coef)

In [ ]:
 
 # by R. Chandra
 #Source: https://github.com/rohitash-chandra/logistic_regression

from math import exp
import numpy as np
import random

SIGMOID = 1
STEP = 2
LINEAR = 3

 
random.seed()

class logistic_regression:

    # num_epocs 迭代次数
    # train_data 训练集
    # test_data 测试集
    # num_features 特征数量
    # learn_rate 学习率
	def __init__(self, num_epocs, train_data, test_data, num_features, learn_rate):
		self.train_data = train_data
		self.test_data = test_data 
		self.num_features = num_features
        # 输出数据维度，列数减去输入特征数量
		self.num_outputs = self.train_data.shape[1] - num_features
        # 样本量
		self.num_train = self.train_data.shape[0]
		self.w = np.random.uniform(-0.5, 0.5, num_features)  # in case one output class 
		self.b = np.random.uniform(-0.5, 0.5, self.num_outputs) 
		self.learn_rate = learn_rate
		self.max_epoch = num_epocs
		self.use_activation = SIGMOID #SIGMOID # 1 is  sigmoid , 2 is step, 3 is linear 
        # 用于记录偏导数
		self.out_delta = np.zeros(self.num_outputs)

		print(self.w, ' self.w init') 
		print(self.b, ' self.b init') 
		print(self.out_delta, ' outdel init')


    # 根据非激活值计算激活值
	def activation_func(self,z_vec):
		if self.use_activation == SIGMOID:
			y =  1 / (1 + np.exp(z_vec)) # sigmoid/logistic
		elif self.use_activation == STEP:
			y = (z_vec > 0).astype(int) # if greater than 0, use 1, else 0
			#https://stackoverflow.com/questions/32726701/convert-real-valued-numpy-array-to-binary-array-by-sign
		else:
			y = z_vec
		return y
 
    # 计算y的输出值
	def predict(self, x_vec ): 
		z_vec = x_vec.dot(self.w) - self.b 
		output = self.activation_func(z_vec) # Output  
		return output
	
	# 计算梯度 （已修正）
    # 预测值减实际值：梯度， 实际值减预测值：负梯度
	def gradient(self, x_vec, output, actual):   
		if self.use_activation == SIGMOID :
			out_delta =   (output - actual)*(output*(1-output)) 
		else: # for linear and step function  
			out_delta =   (output - actual) 
		return out_delta

	# 更新参数：这里有问题，正确写法是改成-=
	def update(self, x_vec, output, actual):      
		self.w+= self.learn_rate *( x_vec *  self.out_delta)
		self.b+=  (1 * self.learn_rate * self.out_delta)
 
	# 计算 SSE
    # 对于分类问题，这里的计算 将概率与实际值之差 视为伪残差
	def squared_error(self, prediction, actual):
		return  np.sum(np.square(prediction - actual))/prediction.shape[0]# to cater more in one output/class
 

	# 评估模型
	def test_model(self, data, tolerance):  

		num_instances = data.shape[0]

		class_perf = 0
		sum_sqer = 0   
        
        #循环遍历每一个样本
		for s in range(0, num_instances):	

			input_instance  =  self.train_data[s,0:self.num_features] 
			actual  = self.train_data[s,self.num_features:]  
			prediction = self.predict(input_instance)
			sum_sqer += self.squared_error(prediction, actual)

			# 设置分类概率阈值
			pred_binary = np.where(prediction > (1 - tolerance), 1, 0)

			print(s, actual, prediction, pred_binary, sum_sqer, ' s, actual, prediction, sum_sqer')

 
			# 预测正确 +1
			if( (actual==pred_binary).all()):
				class_perf =  class_perf +1   

		rmse = np.sqrt(sum_sqer/num_instances)

		percentage_correct = float(class_perf/num_instances) * 100 

		print(percentage_correct, rmse,  ' class_perf, rmse') 
		# note RMSE is not a good measure for multi-class probs

		return ( rmse, percentage_correct)





 
	def SGD(self):   
		
			epoch = 0 
			shuffle = True

			while  epoch < self.max_epoch:
				sum_sqer = 0
				for s in range(0, self.num_train): 

					if shuffle ==True:
						i = random.randint(0, self.num_train-1)

					input_instance  =  self.train_data[i,0:self.num_features]  
					actual  = self.train_data[i,self.num_features:]  
					prediction = self.predict(input_instance) 
					sum_sqer += self.squared_error(prediction, actual)
					self.out_delta = self.gradient( input_instance, prediction, actual)    # major difference when compared to GD
					#print(input_instance, prediction, actual, s, sum_sqer)
                    
                    # 使用梯度，更新参数
					self.update(input_instance, prediction, actual)

			
				print(epoch, sum_sqer, self.w, self.b)
				epoch=epoch+1  

			rmse_train, train_perc = self.test_model(self.train_data, 0.3) 
			rmse_test =0
			test_perc =0
			#rmse_test, test_perc = self.test_model(self.test_data, 0.3)
  
			return (train_perc, test_perc, rmse_train, rmse_test) 
				
	# 梯度下降
	def GD(self):   
		
			epoch = 0 
			while  epoch < self.max_epoch:
				sum_sqer = 0
				for s in range(0, self.num_train): 
					input_instance  =  self.train_data[s,0:self.num_features]  
					actual  = self.train_data[s,self.num_features:]   
					prediction = self.predict(input_instance) 
					sum_sqer += self.squared_error(prediction, actual) 
					self.out_delta+= self.gradient( input_instance, prediction, actual)    # this is major difference when compared with SGD

					#print(input_instance, prediction, actual, s, sum_sqer)
                
					# 使用梯度，更新参数
					self.update(input_instance, prediction, actual)

			
				print(epoch, sum_sqer, self.w, self.b)
				epoch=epoch+1  

			rmse_train, train_perc = self.test_model(self.train_data, 0.3) 
			rmse_test =0
			test_perc =0
			#rmse_test, test_perc = self.test_model(self.test_data, 0.3)
  
			return (train_perc, test_perc, rmse_train, rmse_test) 
				
	
 

#------------------------------------------------------------------
#MAIN



def main(): 

	random.seed()
	 

	 
	dataset = [[2.7810836,2.550537003,0],
		[1.465489372,2.362125076,0],
		[3.396561688,4.400293529,0],
		[1.38807019,1.850220317,0],
		[3.06407232,3.005305973,0],
		[7.627531214,2.759262235,1],
		[5.332441248,2.088626775,1],
		[6.922596716,1.77106367,1],
		[8.675418651,-0.242068655,1],
		[7.673756466,3.508563011,1]]


	train_data = np.asarray(dataset) # convert list data to numpy
	test_data = train_data

	 

	learn_rate = 0.3
	num_features = 2
	num_epocs = 20

	print(train_data)
	 

	lreg = logistic_regression(num_epocs, train_data, test_data, num_features, learn_rate)
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.SGD()
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.GD() 
	 

	#-------------------------------
	#xor data


	xor_dataset= [[0,0,0],
		[0,1,1],
		[1,0,1],
		[1,1,0] ]

	xor_data = np.asarray(xor_dataset) # convert list data to numpy



	num_epocs = 20
	learn_rate = 0.9
	num_features = 2

	lreg = logistic_regression(num_epocs, xor_data, xor_data, num_features, learn_rate)
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.SGD()
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.GD() 


if __name__ == "__main__": main()

In [ ]:
# LogLoss计算

import numpy as np
# 对数损失计算函数
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

# 下面是例子：用这个函数做了两个计算
h= np.random.rand(5)
print(h, ' h')
y= np.random.rand(5)
print(y, ' y')
  
log_loss = loss(h, y) # case of regression or prediction problem
print(log_loss)

y_ = np.ones(5)
print(y_, ' y_')
log_loss = loss(h, y_) # case of classification problem (assume class one 1s)
print(log_loss)

# Week 9

In [ ]:
# SVD数学计算代码

from numpy import array
from numpy import diag
from numpy import zeros
from scipy.linalg import svd
# define a matrix
A = array([
	[1,2,3,4,5,6,7,8,9,10],
	[11,12,13,14,15,16,17,18,19,20],
	[21,22,23,24,25,26,27,28,29,30]])
print(A)
# Singular-value decomposition
# A = USV (S为sigma,但是表示为一个向量而非矩阵，包含了sigma矩阵的全部对角元素)
U, s, VT = svd(A)

# 使用对角元素向量s构建sigma矩阵
# create m x n Sigma matrix
Sigma = zeros((A.shape[0], A.shape[1]))
# populate Sigma with n x n diagonal matrix
Sigma[:A.shape[0], :A.shape[0]] = diag(s)

# 特征选择，选择sigma的前两列，V的前两行
# select
n_elements = 2
Sigma = Sigma[:, :n_elements]
VT = VT[:n_elements, :]

# 点乘，B是原矩阵A的一个低秩近似
# reconstruct
B = U.dot(Sigma.dot(VT))
print(B)

# SVD分解结果为T，有两种计算方法，结果一致
# transform
T = U.dot(Sigma)
print(T)
T = A.dot(VT.T)
print(T)

#Running the example first prints the defined matrix then the reconstructed
# approximation, followed by two equivalent transforms of the original matrix.

print(' Truncated SVD')
#Truncated SVD

#The TruncatedSVD class can be created in which you must specify the number
# of desirable features or components to select, e.g. 2. Once created, 
#you can fit the transform (e.g. calculate V^Tk) by calling the fit() 
#function, then apply it to the original matrix by calling the transform() 
#function. The result is the transform of A called T above.

from sklearn.decomposition import TruncatedSVD
# define array
A = array([
	[1,2,3,4,5,6,7,8,9,10],
	[11,12,13,14,15,16,17,18,19,20],
	[21,22,23,24,25,26,27,28,29,30]])
print(A)
# svd
svd = TruncatedSVD(n_components=2)
svd.fit(A)
result = svd.transform(A)
print(result)

In [ ]:
# 同理，只是这段代码用了np
import numpy as np
U, s, Vt = np.linalg.svd(A)
W2 = Vt.T[:, :2]
X2D = A.dot(W2)

In [ ]:
# 增量PCA

# Authors: Kyle Kastner
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, IncrementalPCA

# 加载数据
iris = load_iris()
X = iris.data
y = iris.target

n_components = 2
ipca = IncrementalPCA(n_components=n_components, batch_size=10)
X_ipca = ipca.fit_transform(X)

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

print(X_ipca, ' X_ipca')

colors = ['navy', 'turquoise', 'darkorange']

for X_transformed, title in [(X_ipca, "Incremental PCA"), (X_pca, "PCA")]:
    plt.figure(figsize=(8, 8))
    for color, i, target_name in zip(colors, [0, 1, 2], iris.target_names):
        plt.scatter(X_transformed[y == i, 0], X_transformed[y == i, 1],
                    color=color, lw=2, label=target_name)

    if True or "Incremental" in title:
        err = np.abs(np.abs(X_pca) - np.abs(X_ipca)).mean()
        plt.title(title + " of iris dataset\nMean absolute unsigned error "
                  "%.6f" % err)
    else:
        plt.title(title + " of iris dataset")
    plt.legend(loc="best", shadow=False, scatterpoints=1)
    plt.axis([-4, 4, -1.5, 1.5])

plt.savefig('ipca.png')


In [ ]:
#K-Means 聚类算法

import matplotlib.pyplot as plt
from matplotlib import style
#style.use('ggplot')
import numpy as np


class K_Means:
    def __init__(self, k=3, tol=0.01, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

        self.centroids = {}

    def fit(self,data):


        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            print(i, self.centroids, ' centroids')
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []
            
            print(self.classifications, i, ' * ')

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids] # check distance of every data point to every centroid
                print(distances, featureset, ' dist - featureset')
                classification = distances.index(min(distances)) # get index of centroid that best suits the data point
                self.classifications[classification].append(featureset) # assign the centroid id to the data point
            
            print(self.classifications, i, ' + ')

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids: # update centroid position
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False

            if optimized:
                break
        return self.centroids

    def predict(self,data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

#main

X = np.array([[1, 2],
              [1.5, 1.8],
              [5, 8 ],
              [8, 8],
              [1, 0.6],
              [9,11],
              [1,3],
              [8,9],
              [0,3],
              [5,4],
              [6,4],])

plt.scatter(X[:,0], X[:,1], s=150)
plt.savefig('data.png')
print(X, ' data')

colors = 10*["g","r","c","b","k"]
clf = K_Means()
x = clf.fit(X)
print(x, 'final centroids')

for centroid in clf.centroids:
    plt.scatter(clf.centroids[centroid][0], clf.centroids[centroid][1],
                marker="o", color="k", s=150, linewidths=5)

for classification in clf.classifications:
    color = colors[classification]
    for featureset in clf.classifications[classification]:
        plt.scatter(featureset[0], featureset[1], marker="x", color=color, s=150, linewidths=5)

# in case we wish to test if data belongs to a cluster

##unknowns = np.array([[1,3],
##                     [8,9],
##                     [0,3],
##                     [5,4],
##                     [6,4],])
##
##for unknown in unknowns:
##    classification = clf.predict(unknown)
##    plt.scatter(unknown[0], unknown[1], marker="*", color=colors[classification], s=150, linewidths=5)
##

plt.savefig('kmeans.png')

In [ ]:
# 软聚类代码

# Common imports
import numpy as np
import os
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
# Let's start by generating some blobs ---
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)
# ------- end of generating blobs ----- 
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

# The following assigns new instances to the cluster whose centroid is closest
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])

print('kmeans.transform(X_new):')
print(kmeans.transform(X_new))

